# Lezione 5 — Introduzione a Scikit-Learn: Modelli Lineari e Logistic Regression

---

## Obiettivi della Lezione

Al termine di questa lezione sarai in grado di:

1. Comprendere le **quattro famiglie di modelli** in Scikit-Learn
2. Capire la **matematica dei modelli lineari** (combinazione lineare, sigmoide, logit)
3. Applicare la **pipeline standard ML**: split, scaling, fit, predict, score
4. Implementare una **Logistic Regression** completa
5. Interpretare i **coefficienti** del modello
6. Scegliere il modello appropriato per diversi problemi

---

## Prerequisiti

- Lezione 4: GroupBy e Transform (Pandas avanzato)
- Conoscenza base di NumPy e Pandas
- Concetti matematici: funzioni, esponenziale, logaritmo

---

## Indice

1. **SEZIONE 1** — Teoria Concettuale Approfondita
2. **SEZIONE 2** — Schema Mentale / Mappa Decisionale
3. **SEZIONE 3** — Notebook Dimostrativo
4. **SEZIONE 4** — Metodi Spiegati
5. **SEZIONE 5** — Glossario
6. **SEZIONE 6** — Errori Comuni e Debug Rapido
7. **SEZIONE 7** — Conclusione Operativa
8. **SEZIONE 8** — Checklist di Fine Lezione
9. **SEZIONE 9** — Changelog Didattico

---

## Librerie Utilizzate

```python
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
```

---

# SEZIONE 1 — Teoria Concettuale Approfondita

## 1.1 Glossario Preliminare

Prima di leggere qualsiasi formula, fissiamo un glossario chiaro:

| Termine | Definizione |
|---------|-------------|
| **Feature** | Variabile di input che descrive un'osservazione (es. eta, reddito) |
| **Target** | Variabile da predire (output del modello) |
| **Modello lineare** | Combina le feature con pesi e bias per produrre un output |
| **Logit** | Trasformazione $\log \frac{p}{1-p}$ che converte probabilita in quantita illimitata |
| **Sigmoid** | Funzione $\sigma(z) = \frac{1}{1 + e^{-z}}$ che schiaccia valori tra 0 e 1 |
| **Odds** | Rapporto tra probabilita di successo e insuccesso: $\frac{p}{1-p}$ |
| **Regularizzazione** | Penalita sui pesi per ridurre overfitting |

## 1.2 Le Quattro Famiglie di Modelli

Scikit-Learn offre molti modelli, ma pensarli per **famiglie concettuali** aiuta a scegliere:

### Famiglia 1: Modelli Lineari
- **Esempi**: `LinearRegression`, `LogisticRegression`, `Ridge`, `Lasso`
- **Idea**: stimano una combinazione lineare delle feature
- **Formula**: $\hat{y} = w_1 x_1 + w_2 x_2 + \dots + w_n x_n + b$
- **Pro**: interpretabili, veloci, ottime baseline
- **Contro**: assumono relazioni lineari

### Famiglia 2: Tree-Based (Alberi)
- **Esempi**: `DecisionTreeClassifier`, `RandomForestClassifier`, `GradientBoostingClassifier`
- **Idea**: suddivisione dello spazio tramite soglie
- **Pro**: catturano non-linearita, robusti
- **Contro**: rischio overfitting, meno interpretabili

### Famiglia 3: Distanza / Vicinanza (KNN)
- **Esempi**: `KNeighborsClassifier`, `KNeighborsRegressor`
- **Idea**: classificano in base ai vicini piu simili
- **Pro**: nessuna assunzione forte, intuitivi
- **Contro**: lenti in predizione, sensibili allo scaling

### Famiglia 4: Probabilistici / Bayesiani
- **Esempi**: `GaussianNB`, `MultinomialNB`
- **Idea**: applicano il teorema di Bayes
- **Pro**: velocissimi, ottimi su dati testuali
- **Contro**: assumono indipendenza tra feature

## 1.3 Matematica della Logistic Regression

### Step 1: Combinazione Lineare (Logit)
$$z = w_1 x_1 + w_2 x_2 + \dots + w_n x_n + b = \mathbf{w}^T \mathbf{x} + b$$

### Step 2: Funzione Sigmoide
$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

La sigmoide trasforma z (che va da $-\infty$ a $+\infty$) in una probabilita (tra 0 e 1).

### Probabilita Stimata
$$\hat{p} = P(y=1 \mid x) = \sigma(\mathbf{w}^T \mathbf{x} + b)$$

### Interpretazione dei Coefficienti
- $w_j > 0$: aumentando la feature j, aumenta la probabilita della classe 1
- $w_j < 0$: aumentando la feature j, diminuisce la probabilita
- $|w_j|$ grande: la feature ha forte impatto

## 1.4 Perche Standardizzare?

**StandardScaler** applica:
$$x' = \frac{x - \mu}{\sigma}$$

**Benefici:**
- Stabilita numerica
- Convergenza piu rapida
- Coefficienti comparabili tra feature
- **Necessario** per modelli lineari, KNN, SVM

---

# SEZIONE 2 — Schema Mentale / Mappa Decisionale

## Pipeline Standard ML

```
[Dati Grezzi]
      |
      v
[Feature Engineering]  ← Crea/trasforma variabili
      |
      v
[Train/Test Split]     ← Dividi per validazione
      |
      v
[Preprocessing]        ← Scaling, encoding
      |
      v
[Fit del Modello]      ← Apprendimento
      |
      v
[Predict]              ← Previsioni su test
      |
      v
[Score/Valutazione]    ← Metriche
      |
      v
[Iterazione]           ← Migliora se necessario
```

## Come Scegliere un Modello?

```
TIPO DI PROBLEMA?
       |
   ┌───┴───┐
   v       v
CLASSIF. REGRESS.
   |       |
   v       v
Quante classi?  Target continuo?
   |              |
   v              v
BINARIA → LogisticRegression, RandomForest
MULTI   → Stessi + SoftMax
CONTINUO → LinearRegression, RandomForest
```

## Euristiche di Scelta Rapida

| Situazione | Modello Consigliato |
|------------|---------------------|
| Dati quasi lineari | Logistic/Linear Regression |
| Non-linearita sospette | RandomForest, GradientBoosting |
| Dataset enorme | GradientBoosting, SGDClassifier |
| Testo (NLP classico) | Naive Bayes + TF-IDF |
| Dataset piccolo | KNN |
| Serve interpretabilita | Logistic Regression, Decision Tree |

## Checklist Pre-Modellazione

1. Ho definito chiaramente il target?
2. Ho separato train e test PRIMA di qualsiasi preprocessing?
3. Ho standardizzato le feature (per modelli lineari)?
4. Ho verificato il bilanciamento delle classi?
5. Ho scelto le metriche appropriate?

---

# SEZIONE 3 — Notebook Dimostrativo

## 3.1 Setup e Creazione Dataset Sintetico

**Perche questo passaggio:** Usiamo `make_classification` per creare un dataset controllato. Questo ci permette di capire il flusso senza preoccuparci della qualita dei dati reali.

**Parametri chiave:**
- `n_samples`: numero di osservazioni
- `n_features`: numero di feature
- `n_informative`: feature effettivamente utili
- `class_sep`: separabilita delle classi (piu alto = piu facile)

In [ ]:
# === SETUP: Import librerie e creazione dataset sintetico ===

import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Creiamo un dataset sintetico per classificazione binaria
X, y = make_classification(
    n_samples=600,       # 600 osservazioni
    n_features=6,        # 6 feature
    n_informative=4,     # 4 feature informative
    n_redundant=0,       # 0 feature ridondanti
    class_sep=1.5,       # Buona separabilita
    random_state=42      # Riproducibilita
)

# --- MICRO-CHECKPOINT ---
print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")
print(f"Classi uniche: {np.unique(y)}")
print(f"Bilanciamento classi: {y.mean():.2f} (0.5 = perfettamente bilanciato)")

assert X.shape == (600, 6), "Shape X non corretta!"
assert y.shape == (600,), "Shape y non corretta!"
print("\n--- Micro-checkpoint: dataset creato correttamente ---")

Shape X: (600, 6)
Positive class ratio: 0.50


---

## 3.2 Train/Test Split

**Perche questo passaggio:** Dividiamo i dati in training (75%) e test (25%). Il test set serve per valutare il modello su dati MAI visti durante l'addestramento.

**Parametri chiave:**
- `test_size=0.25`: 25% per il test
- `stratify=y`: mantiene le proporzioni delle classi
- `random_state=42`: riproducibilita

In [ ]:
# === TRAIN/TEST SPLIT ===

# Dividiamo i dati: 75% training, 25% test
# stratify=y mantiene le proporzioni delle classi in entrambi i set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.25, 
    stratify=y, 
    random_state=42
)

# --- MICRO-CHECKPOINT ---
print(f"Training set: {X_train.shape[0]} campioni")
print(f"Test set: {X_test.shape[0]} campioni")
print(f"Proporzione classe 1 in train: {y_train.mean():.2f}")
print(f"Proporzione classe 1 in test: {y_test.mean():.2f}")

# Verifica stratificazione
assert abs(y_train.mean() - y_test.mean()) < 0.05, "Stratificazione non corretta!"
print("\n--- Micro-checkpoint: split stratificato corretto ---")

(450, 6) (150, 6)


---

## 3.3 Standardizzazione (Scaling)

**Perche questo passaggio:** I modelli lineari sono sensibili alla scala delle feature. Standardizziamo per avere media=0 e std=1.

**IMPORTANTE:** 
- Fit dello scaler SOLO su training set
- Transform su ENTRAMBI train e test
- Mai fare fit sul test set (data leakage!)

In [ ]:
# === STANDARDIZZAZIONE ===

# Creiamo lo scaler
scaler = StandardScaler()

# fit_transform su training: calcola media/std E trasforma
X_train_scaled = scaler.fit_transform(X_train)

# SOLO transform su test: usa media/std del training
X_test_scaled = scaler.transform(X_test)

# --- MICRO-CHECKPOINT ---
print("Medie delle feature dopo scaling (training):")
print(f"  {X_train_scaled.mean(axis=0).round(10)}")
print("\nDeviazioni standard dopo scaling (training):")
print(f"  {X_train_scaled.std(axis=0).round(2)}")

# Dopo standardizzazione, media deve essere ~0 e std ~1
assert np.allclose(X_train_scaled.mean(axis=0), 0, atol=1e-10), "Media non zero!"
assert np.allclose(X_train_scaled.std(axis=0), 1, atol=0.01), "Std non 1!"
print("\n--- Micro-checkpoint: scaling corretto (media=0, std=1) ---")

[ 7.35214359e-17 -4.49455288e-16  3.50337043e-17  2.91125149e-17
  5.21188031e-16 -1.48029737e-18]


---

## 3.4 Fit del Modello (Addestramento)

**Perche questo passaggio:** Addestriamo la Logistic Regression sui dati di training. Il modello impara i coefficienti ottimali per classificare.

**Interpretazione dei coefficienti:**
- Coefficiente positivo → feature aumenta la probabilita della classe 1
- Coefficiente negativo → feature diminuisce la probabilita
- Coefficiente vicino a 0 → feature poco influente

In [ ]:
# === FIT DEL MODELLO ===

# Creiamo e addestriamo la Logistic Regression
model = LogisticRegression(max_iter=1000)  # max_iter per garantire convergenza
model.fit(X_train_scaled, y_train)

# Estraiamo i coefficienti
coeff = pd.Series(
    model.coef_.ravel(), 
    index=[f"feature_{i}" for i in range(6)]
)

# --- MICRO-CHECKPOINT ---
print("Coefficienti del modello (ordinati per importanza):")
print(coeff.sort_values(ascending=False))
print(f"\nIntercetta (bias): {model.intercept_[0]:.4f}")

# Verifica che il modello sia stato addestrato
assert hasattr(model, 'coef_'), "Modello non addestrato!"
print("\n--- Micro-checkpoint: modello addestrato correttamente ---")

feature_1    1.158654
feature_4    0.782303
feature_5    0.598550
feature_2    0.096979
feature_3    0.065306
feature_0   -0.824411
dtype: float64

---

## 3.5 Predict e Valutazione

**Perche questo passaggio:** Usiamo il modello addestrato per fare previsioni sul test set e valutiamo la performance con l'accuracy.

**Accuracy:** proporzione di previsioni corrette
$$\text{Accuracy} = \frac{\text{Previsioni Corrette}}{\text{Totale Previsioni}}$$

In [ ]:
# === PREDICT E VALUTAZIONE ===

# Previsioni sul test set
y_pred = model.predict(X_test_scaled)

# Calcoliamo l'accuracy
acc = accuracy_score(y_test, y_pred)

# --- MICRO-CHECKPOINT ---
print(f"Previsioni effettuate: {len(y_pred)}")
print(f"Distribuzione previsioni: classe 0 = {(y_pred == 0).sum()}, classe 1 = {(y_pred == 1).sum()}")
print(f"\nAccuracy sul test set: {acc:.3f} ({acc*100:.1f}%)")

# Verifica che l'accuracy sia ragionevole (sopra il caso random 50%)
assert acc > 0.5, "Accuracy troppo bassa!"
print("\n--- Micro-checkpoint: accuracy > 50% (meglio del caso) ---")

# Bonus: mostriamo anche le probabilita
y_proba = model.predict_proba(X_test_scaled)
print(f"\nEsempio probabilita (prime 5 osservazioni):")
print(f"  P(classe=0)  P(classe=1)")
for i in range(5):
    print(f"  {y_proba[i, 0]:.3f}        {y_proba[i, 1]:.3f}  → Predetto: {y_pred[i]}")

Accuracy: 0.727


---

# SEZIONE 4 — Metodi Spiegati

## Creazione Dataset

| Metodo | Sintassi | Cosa Fa |
|--------|----------|---------|
| `make_classification()` | `make_classification(n_samples, n_features, ...)` | Genera dataset sintetico |

## Train/Test Split

| Metodo | Sintassi | Cosa Fa |
|--------|----------|---------|
| `train_test_split()` | `train_test_split(X, y, test_size=0.25)` | Divide dati in train e test |
| Parametro `stratify` | `stratify=y` | Mantiene proporzioni classi |
| Parametro `random_state` | `random_state=42` | Riproducibilita |

## Preprocessing

| Metodo | Sintassi | Cosa Fa |
|--------|----------|---------|
| `StandardScaler()` | `scaler = StandardScaler()` | Crea oggetto scaler |
| `.fit_transform()` | `scaler.fit_transform(X_train)` | Calcola parametri E trasforma |
| `.transform()` | `scaler.transform(X_test)` | Solo trasforma (usa parametri esistenti) |

## Modello

| Metodo | Sintassi | Cosa Fa |
|--------|----------|---------|
| `LogisticRegression()` | `model = LogisticRegression()` | Crea modello |
| `.fit()` | `model.fit(X_train, y_train)` | Addestra il modello |
| `.predict()` | `model.predict(X_test)` | Predice le classi |
| `.predict_proba()` | `model.predict_proba(X_test)` | Predice probabilita |
| `.coef_` | `model.coef_` | Coefficienti appresi |
| `.intercept_` | `model.intercept_` | Intercetta (bias) |

## Valutazione

| Metodo | Sintassi | Cosa Fa |
|--------|----------|---------|
| `accuracy_score()` | `accuracy_score(y_true, y_pred)` | Proporzione corrette |

---

# SEZIONE 5 — Glossario

| Termine | Definizione |
|---------|-------------|
| **Feature** | Variabile di input (colonna X) usata per la previsione |
| **Target** | Variabile da predire (y) |
| **Training Set** | Dati usati per addestrare il modello |
| **Test Set** | Dati usati per valutare il modello (mai visti in training) |
| **Fit** | Processo di addestramento del modello |
| **Predict** | Generazione di previsioni dal modello addestrato |
| **Standardizzazione** | Trasformazione per avere media=0 e std=1 |
| **Data Leakage** | Uso improprio di informazioni del test nel training |
| **Accuracy** | Proporzione di previsioni corrette |
| **Coefficiente** | Peso assegnato a ogni feature dal modello |
| **Intercetta (Bias)** | Termine costante del modello |
| **Sigmoide** | Funzione che trasforma valori in probabilita [0,1] |
| **Logit** | Logaritmo degli odds, output lineare prima della sigmoide |
| **Stratificazione** | Mantenere le proporzioni delle classi nel split |
| **Overfitting** | Modello troppo adattato ai dati di training |
| **Regularizzazione** | Tecnica per prevenire overfitting |

---

# SEZIONE 6 — Errori Comuni e Debug Rapido

## Errore 1: Fit dello Scaler sul Test Set (Data Leakage!)

```python
# SBAGLIATO - data leakage!
scaler.fit_transform(X_test)  # NO! Usa info del test

# CORRETTO
scaler.fit_transform(X_train)  # Fit solo su train
scaler.transform(X_test)       # Solo transform su test
```

## Errore 2: Dimenticare di Scalare il Test Set

```python
# SBAGLIATO - test non scalato!
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test)  # X_test non scalato!

# CORRETTO
y_pred = model.predict(X_test_scaled)
```

## Errore 3: Confondere `.predict()` e `.predict_proba()`

```python
# .predict() restituisce le CLASSI (0 o 1)
y_pred = model.predict(X_test)  # [0, 1, 1, 0, ...]

# .predict_proba() restituisce le PROBABILITA
y_proba = model.predict_proba(X_test)  # [[0.3, 0.7], [0.8, 0.2], ...]
```

## Errore 4: Non Impostare `random_state`

```python
# SBAGLIATO - risultati non riproducibili
X_train, X_test, y_train, y_test = train_test_split(X, y)

# CORRETTO - sempre riproducibile
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
```

## Errore 5: Split Prima del Preprocessing? NO!

```python
# SBAGLIATO - ordine errato
X_scaled = scaler.fit_transform(X)  # Scala TUTTO prima
X_train, X_test = train_test_split(X_scaled)  # Poi split

# CORRETTO - split PRIMA, poi scala
X_train, X_test = train_test_split(X)  # Prima split
X_train_scaled = scaler.fit_transform(X_train)  # Poi scala train
X_test_scaled = scaler.transform(X_test)  # Poi scala test
```

## Errore 6: Ignorare ConvergenceWarning

```python
# WARNING: ConvergenceWarning: lbfgs failed to converge

# SOLUZIONE: aumenta max_iter
model = LogisticRegression(max_iter=1000)  # Default e 100
```

## Errore 7: Usare Accuracy su Classi Sbilanciate

```python
# Se 95% dei dati e classe 0, accuracy=0.95 e ingannevole!
# Usa altre metriche:
from sklearn.metrics import precision_score, recall_score, f1_score
```

---

# SEZIONE 7 — Conclusione Operativa

## Cosa Hai Imparato

In questa lezione hai acquisito le basi del Machine Learning con Scikit-Learn:

1. **Quattro famiglie di modelli**: lineari, tree-based, distanza, bayesiani
2. **Pipeline ML standard**: split → scale → fit → predict → score
3. **Logistic Regression**: modello lineare per classificazione binaria
4. **Standardizzazione**: necessaria per modelli lineari
5. **Interpretazione coefficienti**: segno e magnitudo
6. **Metriche**: accuracy come prima metrica

## Pattern Ricorrente: Pipeline di Classificazione

```python
# 1. Split dei dati
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# 2. Scaling (fit solo su train!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Addestramento
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# 4. Previsione e valutazione
y_pred = model.predict(X_test_scaled)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
```

## Collegamento alla Prossima Lezione

Nella Lezione 6 approfondiremo:
- Altre metriche (precision, recall, F1)
- Confusion matrix
- Modelli tree-based

---

# SEZIONE 8 — Checklist di Fine Lezione

Prima di procedere alla prossima lezione, verifica di saper fare tutto quanto segue:

- [ ] Conosco le quattro famiglie di modelli (lineari, tree, distanza, bayesiani)
- [ ] So quando usare un modello lineare vs tree-based
- [ ] So usare `train_test_split()` con `stratify` e `random_state`
- [ ] Capisco perche lo scaling e necessario per modelli lineari
- [ ] So usare `StandardScaler` correttamente (fit su train, transform su entrambi)
- [ ] So creare e addestrare una `LogisticRegression`
- [ ] Capisco la differenza tra `.predict()` e `.predict_proba()`
- [ ] So interpretare i coefficienti del modello (segno e magnitudo)
- [ ] So calcolare l'accuracy con `accuracy_score()`
- [ ] Capisco cos'e il data leakage e come evitarlo
- [ ] So che l'ordine corretto e: split → scale → fit → predict
- [ ] Ricordo di impostare `random_state` per riproducibilita

**Se hai dubbi su qualche punto, rileggi la sezione corrispondente prima di proseguire.**

---

# SEZIONE 9 — Changelog Didattico

| Versione | Data       | Modifiche                                                                 |
|----------|------------|---------------------------------------------------------------------------|
| 1.0      | Originale  | Versione iniziale del notebook (teoria estesa, poco codice)              |
| 2.0      | 2025-12-30 | Ristrutturazione completa secondo template standard a 9 sezioni          |
|          |            | + Nuovo header con obiettivi, prerequisiti e indice                      |
|          |            | + Consolidata SEZIONE 1: Teoria (4 famiglie, matematica logistic)        |
|          |            | + Nuova SEZIONE 2: Schema mentale / mappa decisionale                    |
|          |            | + Riorganizzata SEZIONE 3: Notebook dimostrativo con spiegazioni         |
|          |            | + Aggiunti "Perche questo passaggio" prima di ogni cella di codice       |
|          |            | + Aggiunti micro-checkpoint con asserzioni e sanity check                |
|          |            | + Aggiunto esempio di predict_proba per vedere probabilita               |
|          |            | + Rimosse sezioni duplicate/verbose                                      |
|          |            | + Aggiunta SEZIONE 4: Metodi spiegati con tabelle di riferimento         |
|          |            | + Aggiunta SEZIONE 5: Glossario (16 termini)                             |
|          |            | + Aggiunta SEZIONE 6: Errori comuni e debug rapido (7 errori)            |
|          |            | + Aggiunta SEZIONE 7: Conclusione operativa                              |
|          |            | + Aggiunta SEZIONE 8: Checklist di fine lezione (12 items)               |
|          |            | + Aggiunta SEZIONE 9: Changelog didattico                                |

---

**Fine della Lezione 5**